<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/archived/load_and_compare_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import json

import pandas as pd
import numpy as np

import json
import re

import time
from tqdm import tqdm


In [2]:
!git clone 'https://github.com/mille055/Rec_Project.git'

Cloning into 'Rec_Project'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 401 (delta 102), reused 126 (delta 76), pack-reused 246
Receiving objects: 100% (401/401), 122.30 MiB | 18.28 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (39/39), done.


In [5]:
kaggle_cleaned = pd.read_pickle('/content/Rec_Project/data/archive/cleaned_podcast2.pkl')

In [6]:
kaggle_cleaned

,podcast_id,itunes_id,author,slug,title,description,average_rating,ratings_count,category
0,f201172864310534ae6b6c4f8ae9b8cc,1000016089,noreply@blogger.com (Richard Ham),rahdo-talks-through,Rahdo Talks Through,"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,"[leisure, games]"
1,3cfef94cfccf27eb504bd7a6c7459636,1000035707,CityPodcast,originall,OriginALL,OriginALL este un show cu oameni inteligenti p...,3.7,3,"[careers, business]"
2,9fc35a81647e391f93211b5d11973c83,1000081327,"Jim Kettner, John Mihaly, and Scott Brown",galaktacus,Galaktacus,The name Galaktacus originates from a youthful...,5.0,20,"[film, tv]"
3,ce2c3a68dda35600671acd32a1c15fc5,1000095644,Kris Baker Dersch,no-extra-words-one-persons-search-for-story,No Extra Words one person's search for story,A flash fiction podcast featuring short storie...,5.0,11,"[performing, arts, books]"
4,aed410e50370beec62b241374abb296b,1000114875,Todd Mizener,the-talking-pictures-podcast,The Talking Pictures Podcast,Photojournalism on the radio...sort of. The ph...,5.0,9,"[arts, visual, news]"
...,...,...,...,...,...,...,...,...,...
75324,84d95344bd4f9e98afbc36f645817220,999860044,Society of Hospital Medicine,before-the-white-coat,Before The White Coat,"On Before The White Coat, SHM CEO and host Lar...",5.0,2,"[fitness, careers, business, medicine, health]"
75325,075747aa236dd5e97bfa7834a6bce607,999882879,OakGroveBaptist,oakgrovebaptist,OakGroveBaptist,We want to help you make a connection - a conn...,4.0,1,"[spirituality, religion, christianity]"
75326,c108b5f6d6972a6e9d900d222a0e6309,999909601,"Keating Dental Arts: Dental, Dentistry, Dentist",dental-up,Dental Up,Dental Up is your source for everything dental...,5.0,21,[education]
75327,3c19556d722b83c643f2d0ff59891ec1,999962739,Sarika D. Mehta,sarika-d-mehta-podcast,Sarika D. Mehta » Podcast,Awesome interviews with fascinating people.,5.0,2,"[news, arts]"


In [8]:
kaggle_cleaned.itunes_id.nunique()

75329

75329 unique itunes_id 

In [12]:
ids = kaggle_cleaned.podcast_id.values.tolist()
lenids = set([len(x) for x in ids])
lenids

{32}

all the podcast_id's are 32 characters long. Not matching well with the reviews dataset so want to see how different.

In [14]:
myscraped = pd.read_pickle('/content/Rec_Project/data/podcast_df_040423.pkl')
myscraped

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id,rating,user
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,RobinFerris
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,1,Pops.99
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,ReddEye81
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,2,Keyta7777
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,4,Okkupent
...,...,...,...,...,...,...,...,...,...,...,...,...
657194,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Monijansand
657195,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,trinityangel13
657196,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Kweenkeys
657197,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,JoyfulJoyfulWOG


# Comparison of the two dataframes

In [15]:
print(kaggle_cleaned.columns)
print(myscraped.columns)

Index(['podcast_id', 'itunes_id', 'author', 'slug', 'title', 'description',
       'average_rating', 'ratings_count', 'category'],
      dtype='object')
Index(['title', 'producer', 'genre', 'description', 'num_episodes',
       'avg_rating', 'num_reviews', 'link', 'episode_descriptions',
       'itunes_id', 'rating', 'user'],
      dtype='object')


In [16]:
# number of unique podcasts
print(kaggle_cleaned.itunes_id.nunique(), myscraped.itunes_id.nunique())

75329 3936


In [18]:
# intersection of itunes_id between the two dataframes
common_ids = set(kaggle_cleaned['itunes_id']).intersection(set(myscraped['itunes_id']))
print(len(common_ids))  # Count common identifier values

1269


In [ ]:
ratings_by_author = myscraped.groupby('user').count().rating
ratings_by_author.value_counts()

1     31522
2      1633
3       330
4       114
5        48
6        29
7        20
8         7
9         5
12        4
18        3
10        3
11        2
14        2
20        1
26        1
15        1
23        1
13        1
Name: rating, dtype: int64

In [ ]:
# number of unique users:
myscraped.user.nunique()

33727

In [ ]:
ratings_by_author

user
 JT # 1           1
 hackasaurus      1
 nita05           1
! bajn baioc s    1
!!??              1
                 ..
🧘🏼💓               1
🧠👁👄🦵🏽👅👁👁👥👴🏿🏹      1
🪐🍄🪐               1
🫠😵‍💫🫥             1
🫶🏽❤️🏐             1
Name: rating, Length: 33727, dtype: int64

Top number of reviews by author:

In [ ]:
my_pod_scraped.user.value_counts()

obacker19          26
Maxcraft430        23
malfoxley          20
Arlie K            18
Suzannekmoses      18
                   ..
Bishop Sandoval     1
Bentley2366         1
Phinsup17           1
D. Cincy            1
Dawn Melody         1
Name: user, Length: 33727, dtype: int64

In [ ]:
my_pod_scraped[my_pod_scraped.user=='Maxcraft430']

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,itunes_id,episode_descriptions,user,rating
316,Life Will Be the Death of Me with Chelsea Handler,iHeartPodcasts,Comedy,Listen to Chelsea Handler's first ever podcast...,117,4.3,4300.0,https://podcasts.apple.com/us/podcast/life-wil...,1462096941,[Chelsea and Catherine are joined by Jen Since...,Maxcraft430,5.0
663,All There Is with Anderson Cooper,CNN,Society & Culture,Anderson Cooper takes us on a deeply personal ...,10,4.8,6600.0,https://podcasts.apple.com/us/podcast/all-ther...,1643163707,[Anderson is pleased to present the first epis...,Maxcraft430,5.0
845,Carolla Classics,PodcastOne,Comedy,Producer Chris Laxamana and Superfan Giovanni ...,250,4.7,1500.0,https://podcasts.apple.com/us/podcast/carolla-...,1454001697,[1. Jessimae Peluso (2014)\n2. Dave Koechner (...,Maxcraft430,5.0
913,"Gee Thanks, Just Bought It‪!‬",Forever35/Caroline Moss,Leisure,Ever bought something you loved SO much that y...,210,4.6,866.0,https://podcasts.apple.com/us/podcast/gee-than...,1488696878,[MARCH 2023:\nHi all! For those of you who are...,Maxcraft430,5.0
981,LadyGang,PodcastOne,Society & Culture,Are you afraid to be alone with your own thoug...,500,4.6,10500.0,https://podcasts.apple.com/us/podcast/ladygang...,1064978212,[RKOBH Morgan Stewart brings her down-to-earth...,Maxcraft430,5.0
1041,Good Inside with Dr. Becky,Dr. Becky Kennedy,Kids & Family,Join clinical psychologist and mom of three Dr...,86,4.9,3100.0,https://podcasts.apple.com/us/podcast/good-ins...,1561689671,[This week Dr. Becky and Megan McNamee of Feed...,Maxcraft430,5.0
1197,For The Love With Jen Hatmaker Podcast,For the Love Podcast,Society & Culture,Welcome to the For the Love Podcast community....,358,4.6,6000.0,https://podcasts.apple.com/us/podcast/for-the-...,1258388821,"[Hey there, For the Love Podcast community! We...",Maxcraft430,5.0
1479,Raised By Ricki with Ricki Lake and Kalen Allen,Lemonada,TV & Film,"In the 1990s, a generation of kids, teens, and...",42,4.7,387.0,https://podcasts.apple.com/us/podcast/raised-b...,1643161359,[Grab a friend and some snacks – we’ll provide...,Maxcraft430,5.0
1548,Someone Knows Something,CBC True Crime,True Crime,Host David Ridgen and investigative journalist...,87,4.6,18100.0,https://podcasts.apple.com/us/podcast/someone-...,1089216339,[Access to abortion has taken on urgency as th...,Maxcraft430,3.0
1714,Hollywood Handbook,Sean Clements & Hayes Davenport,Comedy,Hollywood Handbook is an insider's guide to ac...,513,4.9,3300.0,https://podcasts.apple.com/us/podcast/hollywoo...,720832183,[The Boys welcome back AUBREY PLAZA to help ma...,Maxcraft430,5.0


# Get Kaggle Dataset Reviews

In [19]:
### mount google drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [20]:
import os
import zipfile
# Unzip the data
datapath = '/content/gdrive/MyDrive/AIPI540_RecProject/'

if not os.path.exists('./data'):
    os.makedirs('./data')

zip_ref = zipfile.ZipFile(datapath+'reviews.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [21]:
with open('/content/data/reviews.json') as f:
    reviews_df = pd.DataFrame(json.loads(line) for line in f)
reviews_df.head()

,podcast_id,title,content,rating,author_id,created_at
0,52e3d2c4fab4e80a8bb75ad144671d96,Argh,What the heck is this show? And why can’t I de...,1,a23655ce5565f32,2019-05-01 01:24:01+00
1,52e3d2c4fab4e80a8bb75ad144671d96,Miss gross overrated as interviewer.,The Howard stern interview was horrible. I lea...,1,e5d461c5f91c8d2,2019-06-07 14:25:34+00
2,52e3d2c4fab4e80a8bb75ad144671d96,I love Terri and Fresh Air,People get really pissy about this beautiful f...,5,806cc75442c8f3d,2019-05-15 09:42:53+00
3,52e3d2c4fab4e80a8bb75ad144671d96,Fresh Air subscription changes to Code Switch,I’ve been a subscriber to Fresh Air for over a...,4,3e1e7e8417a3e85,2019-05-06 02:39:10+00
4,52e3d2c4fab4e80a8bb75ad144671d96,Climate change,"Hello, my name is Lily Rudman and I am a fifth...",5,7c680144b582c6d,2019-04-18 00:37:51+00


In [23]:
reviews_df.shape

(5607021, 6)

In [25]:
# length of podcast_id values 
ids = reviews_df.podcast_id.values.tolist()
lenids = set([len(x) for x in ids])
lenids

{32}

In [26]:
# intersection of podcast_id with kaggle_cleaned:
common_podcast_ids = set(kaggle_cleaned['podcast_id']).intersection(set(reviews_df['podcast_id']))
print(len(common_podcast_ids))  # Count common identifier values

27


In [33]:
# compare first 20 in alphabetically order:
first_kaggle_ids = sorted(kaggle_cleaned['podcast_id'].values.tolist())[:20]
print(first_kaggle_ids)

first_review_ids = sorted(reviews_df['podcast_id'].values.tolist())[50:100]
print(first_review_ids)



['00012378dc471b46097bb86e093c32f6', '00016d17dfad66524afdad13891e043d', '000197704505764c13db5b6ab586d0af', '00024f2bdb168f6c950baaa22e1e647a', '0003b68c63630a9fa0270a27214e56e8', '000436b595bf75c9820a4a596047e557', '0005e53ebbd4d31d45d876e6beb07f8d', '0006a3a6732240ec1ad91d3596a7de6f', '00078b21e34f79e45a3df104117b444e', '000848bfe757cde2eb6d816664aab0c0', '000880426f481c1e6a2c3e5b97f09582', '000b1a9b02a271eb06c50066b962e5b6', '000b439736b0479f6c6ed03d4f378e62', '000c2edf4b922013a27942c565777a78', '000c7df6025b3939c5fd714cb33ecb64', '000f372fa26b131fc1c7ad882d523fa7', '0011b059982be1f8796e8b2898a6daaf', '0011f370aaa0490d50ad1b414d781cf8', '0012800db0bfa09856b2de1a0fa15643', '0012dec8fb6d7318cf4de88ba73fd7e0']
['0002739ab804936b976f7da844b3e187', '00032c0921fb969700b95576c8d017cd', '00032fb4da5788bbc149463a2baa6a73', '00032fb4da5788bbc149463a2baa6a73', '00032fb4da5788bbc149463a2baa6a73', '000394248d0562f41da76ed53a140b51', '000394248d0562f41da76ed53a140b51', '000394248d0562f41da76ed53

So it seems that there are not many reviews that match podcast_id with the other kaggle dataset, and there is no itunes_id in the reviews data to otherwise match the reviews another way or with the dataset I scraped...options are to re-scrape reviews somehow or ?? 

Now I am looking at the reviews I scraped in myscraped

Alright, now I'll look at the unprocessed kaggle podcast data to compare with the reviews to see if higher number of matching podcast_ids between the two:

In [34]:
zip_ref = zipfile.ZipFile(datapath+'podcasts.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [35]:
with open('/content/data/podcasts.json') as f:
    kaggle_podcasts_df = pd.DataFrame(json.loads(line) for line in f)
kaggle_podcasts_df.head()

,podcast_id,itunes_id,slug,itunes_url,title,author,description,average_rating,ratings_count,scraped_at
0,d3997f1089be9e45fda4113be39765cb,1000000618,christianity-questions-and-answers,https://podcasts.apple.com/us/podcast/christia...,None,None,None,NaN,None,None
1,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345
2,4e5ce6a47e5d491aac3f588cfb3ece73,1000016800,st-neots-evangelical-church-sermons,https://podcasts.apple.com/us/podcast/st-neots...,None,None,None,NaN,None,None
3,6c476d3dd90c3fe5381153354b326952,1000035657,idiotspeakshow,https://podcasts.apple.com/us/podcast/idiotspe...,IdiotSpeakShow,IdiotSpeakShow,Podcast by IdiotSpeakShow,NaN,None,2019-07-08 06:01:23.683147
4,b9e7fdf5cd545fc8097055e2f2f1d760,1000035702,conciertos-en-el-living,https://podcasts.apple.com/us/podcast/conciert...,None,None,None,NaN,None,None


In [36]:
kaggle_podcasts_df

,podcast_id,itunes_id,slug,itunes_url,title,author,description,average_rating,ratings_count,scraped_at
0,d3997f1089be9e45fda4113be39765cb,1000000618,christianity-questions-and-answers,https://podcasts.apple.com/us/podcast/christia...,None,None,None,NaN,None,None
1,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345
2,4e5ce6a47e5d491aac3f588cfb3ece73,1000016800,st-neots-evangelical-church-sermons,https://podcasts.apple.com/us/podcast/st-neots...,None,None,None,NaN,None,None
3,6c476d3dd90c3fe5381153354b326952,1000035657,idiotspeakshow,https://podcasts.apple.com/us/podcast/idiotspe...,IdiotSpeakShow,IdiotSpeakShow,Podcast by IdiotSpeakShow,NaN,None,2019-07-08 06:01:23.683147
4,b9e7fdf5cd545fc8097055e2f2f1d760,1000035702,conciertos-en-el-living,https://podcasts.apple.com/us/podcast/conciert...,None,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...
2077660,bc746cf4f5e78ed7880f964fcebcbe4d,999969174,smarketsights,https://podcasts.apple.com/us/podcast/smarkets...,None,None,None,NaN,None,None
2077661,c16669b79e093e976b83a8e61270a903,999971688,save-to-the-uttermost,https://podcasts.apple.com/us/podcast/save-to-...,None,None,None,NaN,None,None
2077662,b863eaa678d3de4f5721a7492fd32b3e,999975096,the-skift-podcast,https://podcasts.apple.com/us/podcast/the-skif...,The Skift Podcast,Skift,"Skift editors sit down with creatives, executi...",4.5,19,2019-07-07 23:20:30.301705
2077663,920de5c8d1bb6b3fea7d86ac39473d76,999987741,history-of-the-batman,https://podcasts.apple.com/us/podcast/history-...,None,None,None,NaN,None,None


In [37]:
# intersection of podcast_id between reviews_df and kaggle_podcasts_df:
common_podcast_ids_with_full = set(kaggle_podcasts_df['podcast_id']).intersection(set(reviews_df['podcast_id']))
print(len(common_podcast_ids_with_full))  # Count common identifier values

31


Unfortunately, not much better...


In [38]:
# these are the entries in the kaggle_podcasts_df which share common podcast_id with the reviews_df
common = kaggle_podcasts_df[reviews_df.podcast_id.apply(lambda x: x in (common_podcast_ids_with_full))]
common

<ipython-input-38-ec253f3c24c5>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  common = kaggle_podcasts_df[reviews_df.podcast_id.apply(lambda x: x in (common_podcast_ids_with_full))]


,podcast_id,itunes_id,slug,itunes_url,title,author,description,average_rating,ratings_count,scraped_at
48160,edc4c320ebf9eee4a55653b7d42848fb,1133406206,northern-soundings-alaska-in-conversation,https://podcasts.apple.com/us/podcast/northern...,Northern Soundings: Alaska in Conversation,Robert Hannon,"Discussions with artists, scientists, writers",5.0,2,2019-07-08 19:28:31.557855
81017,c30362e36fd6746751ade27e5395dee7,120907544,skatecast,https://podcasts.apple.com/us/podcast/skatecas...,None,None,None,NaN,None,None
81018,f5f7f5721e5516da4016506c88d0377c,1209080762,real-lives-real-stories,https://podcasts.apple.com/us/podcast/real-liv...,None,None,None,NaN,None,None
81019,3b3b1e6b64ef8e8e9d16760ae88c4ef0,1209081933,city-soundtracks,https://podcasts.apple.com/us/podcast/city-sou...,City Soundtracks,Google Play Music,City Soundtracks by Google Play Music is a tra...,4.8,11,2019-07-08 22:29:58.929869
111422,2ad47ab1f6e0f1eb860bb6bc6fd196e3,1268418534,tenerysmithinnovations-show,https://podcasts.apple.com/us/podcast/tenerysm...,None,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...
1434734,52f5b4460ba9c029dba0fa4b366a9f46,1555672726,covid-vaccine,https://podcasts.apple.com/us/podcast/covid-va...,None,None,None,NaN,None,None
1438008,8e8b837650b92ee12ca945659b470f6e,1555950695,sh1tgarbage-movies,https://podcasts.apple.com/us/podcast/sh1tgarb...,None,None,None,NaN,None,None
1438009,384925dcdcf312b0fc159b0da7333e33,1555950950,%E7%B4%AB%E5%BE%AE%E5%90%89%E5%90%89%E5%BA%97,https://podcasts.apple.com/us/podcast/%E7%B4%A...,None,None,None,NaN,None,None
1438010,28afe55956b2d85df4809b3d00a93455,1555950972,inspiring-women-in-business,https://podcasts.apple.com/us/podcast/inspirin...,None,None,None,NaN,None,None


In [40]:
reviews_df.author_id.nunique()

3256501